# **KawsAI - Un Modelo de IA para Matching de ChambeaYa**

En este notebook se guarda el proceso que se hizo para crear el modelo de IA KawsAI. El objetivo de KawsAI, es hacer el matching con la mejor mype a cada estudiante. Así, el estudiante y la mype pueden conectar para poder trabajar juntos.

### **I. Versión:**
1.1

### **II. Objetivo:**
El objetivo de KawsAI es lograr hacer el mejor match entre estudiantes y un puesto ofrecido por una mype. Este match se va a basar en datos que ChambeaYa va a recopilar de ambos lados a traves de su página web. El objetivo principal es lograr indicar a un estudiante cual es el puesto que ofrece una mype que mas coincide con sus conocimientos, y así aumentar mucho las probabilidades de conexión.

### **III. Descripción de los Datasets**
Para la versión 1.0 vamos a usar datasets simulados con la ayuda de la IA de ChatGPT. Se le pidió a ChatGPT simular 2 datasets en forma de tablas y en formato .csv, que son estudiantes_test.csv y mypes_test.csv. Cada una de las columnas de estos datasets contiene información que se va a solicitar a los usuarios ingresar en nuestra plataforma.

##### IIIa. Descripción de columnas de estudiantes:
- **id:** ID único del estudiante (int)
- **areas_interes:** Áreas profesionales de interés del estudiante (string)
- **habilidades_destacadas:** Habilidades más desarrolladas del estudiante (string)
- **main_motivation** Motivación principal del estudiante (string)
- **description:** Descripción libre sobre personalidad y hobbies (string)
- **weekly_availability:** Horas disponibles por semana para trabajar (int)
- **preferred_modality:** Modalidad preferida de trabajo (Presencial:1, Híbrido:2, Remoto:3) (int)
- **experience_description:** Experiencia previa relevante (string)

##### IIIb. Descripción de columnas de puestos de mypes:
- **id:** ID único del puesto ofrecido (string)  
- **title** Título del puesto ofrecido (string)  
- **description:** Descripción detallada del puesto (string)  
- **area_del_puesto:** Área o departamento del puesto (string)  
- **requisitos:** Los requisitos necesarios para el puesto (string)
- **modality:** Modalidad del trabajo (Presencial:1, Híbrido:2, Remoto:3) (int)  
- **required_hours:** Horas semanales requeridas (int)  

### **IV. Índice**
1. Carga de Librerias

## 1. Carga de Librerias

Importamos las librerias mas importantes para nuestro modelo de IA

In [1]:
# Todas las librerias que se deben importar
import re
import pandas as pd
import numpy as np
import spacy
from spacy.lang.es.stop_words import STOP_WORDS as spacy_stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack
from sklearn.neighbors import NearestNeighbors

In [2]:
!python -m spacy download es_core_news_sm

     ---------------------------------------- 0.0/12.9 MB ? eta -:--:--
     --------------------------------------- 0.0/12.9 MB 330.3 kB/s eta 0:00:39
     --------------------------------------- 0.1/12.9 MB 656.4 kB/s eta 0:00:20
     - -------------------------------------- 0.6/12.9 MB 5.0 MB/s eta 0:00:03
     ---- ----------------------------------- 1.5/12.9 MB 8.8 MB/s eta 0:00:02
     -------- ------------------------------- 2.7/12.9 MB 12.2 MB/s eta 0:00:01
     ------------ --------------------------- 3.9/12.9 MB 14.8 MB/s eta 0:00:01
     ---------------- ----------------------- 5.3/12.9 MB 17.0 MB/s eta 0:00:01
     -------------------- ------------------- 6.5/12.9 MB 18.2 MB/s eta 0:00:01
     ------------------------ --------------- 7.9/12.9 MB 19.5 MB/s eta 0:00:01
     ----------------------------- ---------- 9.4/12.9 MB 20.8 MB/s eta 0:00:01
     -------------------------------- ------ 10.9/12.9 MB 27.3 MB/s eta 0:00:01
     ------------------------------------ -- 12.2/


[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


## 2. Carga de datos en DataFrames

In [3]:
# Ahora leemos los csv y los guardamos en dataframes de pandas2
puestos = pd.read_csv("../datasets/puestos_test.csv", index_col=False)

## 3. Preprocesamiento de los datos

### 3.1. Para los estudiantes:

#### La función de preprocesamiento va a hacer diferentes pasos para diferentes columnas:

**1)** Vamos a juntar todas las columnas textuales en una sola para el modelo y se le van a dar un peso a cada columna, luego se aplicará tf-idf a todas estas columnas:
- career (peso de 1)
- habilidades_destacadas (peso de 3)
- areas_interes (peso de 3)
- descripcion_personal (peso de 2)
- experiencia_relevante (peso de 5)

### 3.2.Para los puestos de trabajo:

#### La clase de preprocesamiento va a tener diferentes preprocesamientos para cada columna

**1)** Para las columnas textuales vamos a juntarlo todo como requerimiento_textual, cada columna textual tendrá un peso
- title
- description
- area_del_puesto
- requisitos

In [ ]:
# Se crea una clase preprocesador general
class Preprocesador:
    # Primero al crear la clase, vamos a inciar el TF-IDF Vectorizer y el nlp que es el modelo para la lematización
    def __init__(self):
        self.vectorizador = TfidfVectorizer(max_features=500)
        self.nlp = spacy.load("es_core_news_sm")

    # Con esta función recibimos un texto y lo devolvemos ya lematizado sin las stop_words
    def crear_perfil_textual(self, text):
        text = re.sub(r"[^\w\s]", " ", text.lower())
        text = re.sub(r"\s+", " ", text).strip()
        doc = self.nlp(text)
        tokens = []
        for token in doc:
            if token.is_stop or token.is_punct or token.like_num:
                continue
            else:
                tokens.append(token.lemma_)
        return " ".join(tokens) # Se devuelve un string

    # Función para crear el prefil textual de un estudiante (Pesos para cada tipo de carrera)
    def crear_perfil_textual_estudiante(self, estudiante):
        return self.crear_perfil_textual(
            ("carrera: " + str(estudiante['career']) + " ") * 1 +
            ("habilidades: " + str(estudiante['habilidades_destacadas']) + " ") * 3 +
            ("intereses: " + str(estudiante['areas_interes']) + " ") * 3 +
            ("descripcion: " + str(estudiante['description']) + " ") * 3 +
            ("experiencia: " + str(estudiante['experience_id']) + " ") * 5
        )
    
    # Función para crear el prefil textual de un puesto (Pesos para cada tipo de puesto)
    def crear_perfil_textual_puesto(self, puesto):
        return self.crear_perfil_textual(
            ("puesto: " + str(puesto['title']) + " ") * 1 +
            ("descripcion: " + str(puesto['description']) + " ") * 2 +
            ("area: " + str(puesto['area_id']) + " ") * 1 +
            ("requisitos: " + str(puesto['requisitos'])) * 5
        )

    # Función para realizar la vectorización TF-IDF de los puestos
    def fit_transform(self, puestos):
        # Hacemos una copia de los dataframes para no modificarlos directamente
        puestos = puestos.copy()

        # Creamos el perfil textual para los puestos
        puestos['perfil_textual'] = puestos.apply(self.crear_perfil_textual_puesto, axis=1)

        # Ahora vectorizamos este perfil textual con TF-IDF
        corpus = puestos['perfil_textual']
        tfidf_puestos = self.vectorizador.fit_transform(corpus)

        # Se retorna la matriz vectorizada de los puestos
        return tfidf_puestos
    
    # Funciń para realizar la vectorización TF-IDF de un estudiante
    def transform_estudiante(self, estudiante):
        estudiante = estudiante.copy()
        text = self.crear_perfil_textual_estudiante(estudiante.iloc[0])
        tfidf_estudiante = self.vectorizador.transform([text])

        return tfidf_estudiante

## 4. Función para realizar el primer filtrado de datos
Para el primer filtrado, vamos a filtrar los puestos en base a la modalidad y las horas semanal. 

In [ ]:
def filtrado_por_modalidad_y_horas(X_puestos, puestos, estudiante):
    mod = estudiante['preferred_modality'].iloc[0]
    hrs = estudiante['weekly_availability'].iloc[0]
    mascara = (puestos['modality'] == mod) & (puestos['required_hours'] <= hrs)
    print(mascara.value_counts())
    print(X_puestos.shape[0])  # número de filas en la matriz
    print(len(mascara))        # debería ser igual
    return X_puestos[mascara.values], puestos[mascara.values]

## 5. Creación y Entrenamiento del Modelo
Nuestro modelo va a ser KNN para nuestro MVP, vamos a aplicar primero el entrenamiento del KNN con fit, y luego para cada estudiante solo vamos a usar kneighbors y nos devolvera los indices de los puestos ideales para el estudiante

In [5]:
# Primero creamos la clase para nuestro modelo de IA
class KawsAIModel:
    # Se usa el modelo de knn y se inicializa con 2 principales parametros
    # n_neighbors = El número de mejores match que se va a mostrar
    # metric = El tio de métrica y comparación que se va a usar 
    def __init__(self):
        self.knn = NearestNeighbors(n_neighbors=3, metric='euclidean')
    # Para el entrenamiento se va a usar las matrices TF-IDF de los puestos como parametro y se usará el metodo fit
    def train(self, X_puestos):
        self.knn.fit(X_puestos)
    # Función para obtener los mejores matches, se retornará los índices.
    def get_positions(self, X_estudiante_nuevo):
        distances, indexes = self.knn.kneighbors(X_estudiante_nuevo)
        return indexes[0]

## 6. Funcionamiento del Modelo
Se hará una prueba del modelo para visualizar el funcionamiento 

In [ ]:
# Primero creamos la instancia de nuestro proprocesador y nuestro modelo de ia
pre = Preprocesador()
model = KawsAIModel()

In [105]:
estudiante_nuevo = {
    "carrera": "Ingeniería Industrial",
    "habilidades_destacadas": "Excel avanzado, análisis de procesos, liderazgo, trabajo en equipo, gestión de proyectos",
    "areas_interes": "optimización de procesos, logística, mejora continua, calidad",
    "descripcion_personal": "Soy un estudiante proactivo con interés en mejorar la eficiencia operativa de las empresas. Me gusta trabajar en equipo y resolver problemas complejos.",
    "experiencia_relevante": "Asistente de producción en planta textil, prácticas preprofesionales en área de logística en empresa de alimentos",
    "preferred_modality": "Presencial",
    "weekly_availability": 40
}
estudiante = pd.DataFrame([estudiante_nuevo])

In [64]:
X_puestos = pre.fit_transform(puestos)

In [106]:
X_estudiante_nuevo = pre.transform_estudiante(estudiante)

In [107]:
X_puestos_filtrados, puestos_filtrados = filtrado_por_modalidad_y_horas(X_puestos, puestos, estudiante)

True     369
False    311
Name: count, dtype: int64
680
680


In [108]:
X_puestos_filtrados.shape

(369, 500)

In [109]:
model.train(X_puestos_filtrados)
indices = model.get_positions(X_estudiante_nuevo)

In [110]:
puestos_filtrados.iloc[indices]

,id_puesto,titulo_puesto,descripcion_puesto,area_del_puesto,modalidad_de_trabajo,ubicacion,horas_semanales,horario,idiomas_requeridos,requisitos,fecha_inicio,duracion_del_puesto,fecha_limite_postulacion,tipo_contrato,sueldo_aproximado
417,PSTFAR008,Pasantía en industria cosmética,"Colabora en evaluación de estabilidad, formula...",Farmacia y Bioquímica,Presencial,Lima,25,Fijo,Español,"Conocimiento de emulsiones, cosmética básica, ...",2025-08-15,6.0,2025-08-08,Pasantía,1350.0
220,PSTIAL001,Asistente en Control de Calidad de Alimentos,"Colabora en análisis físico-químicos, muestreo...",Ingeniería de Alimentos,Presencial,Lima,30,Fijo,Español,Buenas Prácticas de Manufactura (BPM); manejo ...,2025-08-01,6.0,2025-07-25,Pasantía,1450.0
123,PSTII014,Asistente de Producción Textil,"Colabora en control de calidad en línea, segui...",Ingeniería Industrial,Presencial,Gamarra,30,Fijo,Español,Conocimiento básico de procesos textiles; Exce...,2025-08-12,6.0,2025-08-06,Pasantía,1350.0
